In [ ]:
# Import Statements
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
from std_msgs.msg import Int32MultiArray as Array
from std_msgs.msg import MultiArrayDimension as Dimension
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String
import EnvMap
import Buoy_List
from numba import jit

In [ ]:
class Buoy:
    def __init__(self):
        self._y = 0
        self._x = 0
        self._color = "none"
        self._green_hits = 0
        self._red_hits = 0
        self._origin_x = 0
        self._origin_y = 0

    def __init__(self, attribute_arr):
        self._y = attribute_arr[0]
        self._x = attribute_arr[1]
        self._color = "none"
        self._green_hits = attribute_arr[2].count("green")
        self._red_hits = attribute_arr[2].count("red")
        self._origin_x = attribute_arr[4]
        self._origin_y = attribute_arr[3]

    def __set_attributes(self, attribute_arr):
        self._y = attribute_arr[0]
        self._x = attribute_arr[1]
        self._origin_x = attribute_arr[4]
        self._origin_y = attribute_arr[3]

    def increment_hits(self, color):
        if color == "green":
            self._green_hits = self._green_hits + 1
        elif color == "red":
            self._red_hits = self._red_hits + 1

    def get_attributes(self):
        if self._red_hits > self._green_hits:
            self._color = "red"
        elif self._green_hits > self._red_hits:
            self._color = "green"
        else:
            self._color = "none"
        return [self._y, self._x, self._color, self._origin_y, self._origin_x]

    def get_hits(self):
        if self._red_hits > self._green_hits:
            return self._red_hits
        elif self._green_hits > self._red_hits:
            return self._green_hits
        return 0


class BuoyList:
    def __init__(self):
        self._buoy_list = []
        self._confirm_thresh = 8

    def add_buoy(self, buoy):
        if len(self._buoy_list) > 0:
            found = False
            for item in self._buoy_list:
                item_attr = item.get_attributes()
                if buoy[0] == item_attr[0] and buoy[1] == item_attr[1]:
                    item.increment_hits(buoy[2])
                    found = True
            if not found:
                self._buoy_list.append(Buoy(buoy))
        else:
            self._buoy_list.append(Buoy(buoy))

    def get_buoys(self):
        return self._buoy_list

    def clear_buoy_list(self):
        self._buoy_list = []

    def set_confirmation_threshold(self, thresh):
        self._confirm_thresh = thresh

    def get_confirmation_threshold(self):
        return self._confirm_thresh

    def get_confirmed_buoys(self):
        arr = []
        for item in self._buoy_list:
            if item.get_hits() > self._confirm_thresh:
                arr.append(item.get_attributes())
        return arr

class EnvMap:
    def __init__(self, image, odom_coords):
        """
        Initializes the class variables and finds any buoys in the given map image

        :param image: A 2D list containing the current data in the map
        :param odom_coords: The position of the odometry reference on the map image
        """
        self._offset = odom_coords
        self._curr_map = image
        self._buoy_data = []
        self.__log_buoys()

    def __log_buoys(self):
        """
        Logs the current position and color of any buoys in the map

        :param image: A 2D list containing the current data in the map
        :return: None
        """
        if np.isin(self._curr_map, [30,50]).any():
            for i, row in enumerate(self._curr_map):
                if np.isin(row, [30,50]).any():
                    for j, pixel in enumerate(row):
                        if pixel == 30:
                            y = i - self._offset[0]
                            x = j - self._offset[1]
                            self._buoy_data.append([y, x, "green", self._offset[0],self._offset[1]])
                        elif pixel == 50:
                            y = i - self._offset[0]
                            x = j - self._offset[1]
                            self._buoy_data.count([y,x,"red"])
                            self._buoy_data.append([y, x, "red", self._offset[0],self._offset[1]])

    def get_map(self):
        """
        Returns the map stored in this instance of the class

        :return: A 2D list containing the current data in the map
        """
        return self._curr_map

    def get_buoys(self):
        """
        Returns the set of all buoys in the form of (y, x, color)

        :return: A list of lists, where each sublist is the coordinates and color of a buoy
        """
        return self._buoy_data

    def set_buoys(self, buoy_list):
        self._buoy_data = buoy_list
        print buoy_list
        if buoy_list is not None:
            for item in buoy_list:
                self._curr_map[item[0]+item[3]+self._offset[0]][item[1]+item[4]+self._offset[1]] = 30 if item[2] == "green" else 50 if "red" else self._curr_map[item[0]+item[3]-self._offset[0]][item[1]+item[4]-self._offset[1]]     
# End of Env_Map Class



In [ ]:
# Initialization Stuff

rospy.init_node('liveproc')

listener = tf.TransformListener()
pub_array = rospy.Publisher('updated_array', Array, queue_size=10)
pub_map = rospy.Publisher('updated_map', Map, queue_size=10)

occ_map = None
up_occ_map = None
map_resolution = 0.5
origin = None
pre_timestep = None
robot_x = 0
robot_y = 0
map_width = 0
b_list = []
tri_width = 5
dist_error = 0.15
map_height = 0
camera_range = 85/map_resolution

new_map_size = False
overlay = True
display_robot_yaw = False
display_rect = True
display_map = True
display_info = False
visualization = False
publish_map_or_nah = True

np.set_printoptions(threshold=sys.maxsize)

In [ ]:
def disp_rect(rect):
    cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
    cv.imshow("stereo/left/image_rect_color", rect)
    cv.waitKey(1)

In [ ]:
def disp_map(map_img):
    map_img = np.swapaxes(map_img, 0, 1)
    cv.namedWindow("map_inverted",cv.WINDOW_NORMAL)
    cv.imshow("map_inverted", map_img)
    cv.waitKey(1)

In [ ]:
def transform(img):

    global bearings
    global areas
    global tags    
    
    bridge = cv_bridge.CvBridge()
    
    areas = []
    bearings = []
    tags = []
    
    width = img.height
    height = img.width
    
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)  
    index_n_img = pp(img,width,height)
    img2= dbscan(img, index_n_img)

    return img2

def pp(image, width, height):

    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    row, col = np.indices((width,height))
    index_n_img = np.vstack(([col.flatten()],[row.flatten()],[image[:,:,0].flatten()],[image[:,:,1].flatten()],[image[:,:,2].flatten()])).T 
    index_n_img = index_n_img[np.logical_or(np.logical_and(index_n_img[:,2] > 70, index_n_img[:,2] < 90),
                                        np.logical_or(index_n_img[:,2] >= 170, index_n_img[:,2] < 10))]
    index_n_img = index_n_img[np.logical_and(index_n_img[:,3] > 5,index_n_img[:,4] > 10)]
    return index_n_img

def dbscan(original, index_n_img):
    if len(index_n_img) > 0:
        db = DBSCAN(eps=3, min_samples=10).fit(index_n_img[:,[0,1]])
        labels = db.labels_
        array = np.column_stack((index_n_img,labels))
        sort = array[np.argsort(array[:, 5])]
        sort = sort[np.logical_not(sort[:,5] == -1)]      
        if len(sort) > 0:
            tb = None
            first_locations = np.delete(np.nonzero(np.r_[1,np.diff(sort[:,5])[:-1]]),0)
            points_split = np.split(sort,np.reshape(first_locations,-1))
            for clusters in np.arange(len(points_split)).tolist():
                tb = rectangle(points_split[clusters],original)
            original = tb
    return original


In [ ]:
def rectangle(mat,original):
    
    rect = cv.boundingRect(mat[:,[0,1]])
    min_rect = cv.minAreaRect(mat[:,[0,1]])

    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    
    tag = 0

    if width < height < 4*width and width*height > 45:
        tag, original = identify_buoy_overlay(mat,original,x,y,width,height,min_rect,tag)
    return original

def identify_buoy_overlay(mat,original,x,y,width,height,min_rect,tag):
    
    global bearings
    global areas
    global tags
    
    font = cv.FONT_HERSHEY_SIMPLEX
    color = color_hsv(mat)
    if color =='green':
        tag = 2
        cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
        tags.append(tag)
    elif color =='red':
        tag = 3
        cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
        tags.append(tag)
    else:
        tag = 1
        cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
        tags.append(tag)
    cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
    bearing = (min_rect[0][0]-640.0)/1280.0*80.0
    cv.putText(original,'bearing: '+ str(round(bearing,2)),(np.int0(x+width/2),np.int0(y+height/2.5)), font, 0.5,(0,0,0),2,cv.LINE_AA)
    bearings.append(bearing)
    area = min_rect[1][0]*min_rect[1][1]
    areas.append(area)
    
    return tag, original
    
def color_hsv(mat):
    
    if len(mat) > 200:
        hue = np.random.choice(mat[:,2],200).sum()/200
        sat= np.random.choice(mat[:,2],200).sum()/200
        val = np.random.choice(mat[:,2],200).sum()/200
    else:
        sum_mat = np.sum(mat,axis=0)
        hue = sum_mat[2]/len(mat)
        sat = sum_mat[3]/len(mat)
        val = sum_mat[4]/len(mat)

    if (hue < 60 or hue >= 170):
        return 'red'
    if (70 < hue < 90):
        return 'green'
    else:
        return hue, sat, val

In [ ]:
def draw_background_objects(map_img, map_np):
    
    start_time = time.time()
    
    global origin
    global robot_x
    global robot_y
    global visualization
    
    o_x = (-origin.x)/map_resolution
    o_y = (-origin.y)/map_resolution
    
# Color Assignment Section

    if visualization:
        map_img[map_np == -1] = [255,255,255] #Change to [255,255,255] for color to [0] for grayscale
        map_img[map_np == 0] = [255,255,255]  #Change to [255,255,255] for color to [0] for grayscale
        map_img[map_np == 100] = [0,0,0]      #Change to [0,0,0] for color to [100] for grayscale
        origin_color = [0,0,255]              #Change to [0,0,255] for color to [0] for grayscale
        robot_color = [0,0,255]               #Change to [0,0,255] for color to [0] for grayscale
    else:
        map_img[map_np == -1] = 0    #Change to [255,255,255] for color to [0] for grayscale
        map_img[map_np == 0] = 0     #Change to [255,255,255] for color to [0] for grayscale
        map_img[map_np == 100] = 100 #Change to [0,0,0] for color to [100] for grayscale
        origin_color = 100           #Change to [0,0,255] for color to [0] for grayscale
        robot_color = 100            #Change to [0,0,255] for color to [0] for grayscale

# Origin (Drawn as a cross)

    map_img[int(o_y)+0][int(o_x)+0] = origin_color
    map_img[int(o_y)+0][int(o_x)+1] = origin_color
    map_img[int(o_y)+0][int(o_x)+2] = origin_color
    map_img[int(o_y)+1][int(o_x)+0] = origin_color
    map_img[int(o_y)+2][int(o_x)+0] = origin_color
    if (-origin.x) != 0:
        map_img[int(o_y)+0][int(o_x)-1] = origin_color
        map_img[int(o_y)+0][int(o_x)-2] = origin_color
    if (-origin.y) != 0:
        map_img[int(o_y)-1][int(o_x)+0] = origin_color
        map_img[int(o_y)-2][int(o_x)+0] = origin_color

# Robot (Drawn as a square)

    map_img[int(robot_y)+0][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+0][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)+2] = robot_color
    map_img[int(robot_y)+0][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)-1][int(robot_x)-2] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)+2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)+2][int(robot_x)-1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+0] = robot_color
    map_img[int(robot_y)-2][int(robot_x)+1] = robot_color
    map_img[int(robot_y)-2][int(robot_x)-1] = robot_color

    return map_img

In [ ]:
def draw_ray(map_img,slices):
    
    start_time = time.time()
    
    global display_robot_yaw
    global bearings
    global tags
    global areas
    global robot_x
    global robot_y
    global map_resolution
    global areas
    global visualization
    
    slices_size = len(slices)
    
    if not display_robot_yaw:
        
        for i in range(int(slices_size)):
            try:
                
                arr = slices[i]
                appr_dist_low = (1.0-2.5*dist_error)*map_resolution*2*456.7946397925*math.pow(areas[i],-0.4705300238)              
                arr_sort = arr[arr[:,3]>appr_dist_low]
                
                # This checks whether there is an occupancy space in the slice
                
                if 100 in arr_sort[:,2]:
                    
                    arr_sort = arr_sort[np.argsort(arr_sort[:,2])]
                    first_locations = np.nonzero(np.r_[1,np.diff(arr_sort[:,2])])
                    points = np.delete(arr_sort,[2,3],axis = 1)
                    points_split = np.split(points,[first_locations[-1][-1]])
                    occ_points = points_split[1]
                    points_split = points_split[0]
                    
                    for os in range(int(len(occ_points))):
                        
                        if tags[i] == 2:
                            if visualization:
                                map_img[occ_points[os][1]][occ_points[os][0]]=[0,255,0]
                            else:
                                map_img[occ_points[os][1]][occ_points[os][0]]=30
                        
                        if tags[i] == 3: 
                            if visualization:
                                map_img[occ_points[os][1]][occ_points[os][0]]=[0,0,255]
                            else:
                                map_img[occ_points[os][1]][occ_points[os][0]]=50

                    # Set to False if you don't want to display the slices

                    if False and visualization:
                        for uos in range(int(len(points_split))):
                            map_img[points_split[uos][1]][points_split[uos][0]]=[255,0,0]
            
            except Exception as ex:
                
                print ex
                
    else:
        
        for arr in slices[-1]:
            
            map_img[arr[1]][arr[0]] = [0,255,0]

    return map_img

In [ ]:
def get_mapnbot_info(trans,robot_yaw):
    start_time = time.time()
    global map_resolution
    global robot_x
    global robot_y
    global map_width
    global map_height
    global origin
    global occ_map
    global map_np
    
    map_array = occ_map.data
    
    map_width = occ_map.info.width
    map_height = occ_map.info.height
    
    map_np = np.reshape(np.array(map_array),(map_height,map_width))

    robot_x = (trans[0]-origin.x)/map_resolution
    robot_y = (trans[1]-origin.y)/map_resolution
    robot_yaw = (90 - robot_yaw)
    while robot_yaw > 360:
        robot_yaw = robot_yaw - 360
    while robot_yaw < 0:
        robot_yaw = robot_yaw + 360
#     print time.time()-start_time, "map/bot info"
    return map_np, robot_yaw

In [ ]:
def insidetriangle((x1,x2,x3),(y1,y2,y3),map_np):
    start_time = time.time()
    global robot_x
    global robot_y
    
    xs=np.array((x1,x2,x3),dtype=float)
    ys=np.array((y1,y2,y3),dtype=float)

    # The possible range of coordinates that can be returned
    x_range=np.arange(np.min(xs),np.max(xs)+1)
    y_range=np.arange(np.min(ys),np.max(ys)+1)

    # Set the grid of coordinates on which the triangle lies. The centre of the
    # triangle serves as a criterion for what is inside or outside the triangle.
    X,Y=np.meshgrid( x_range,y_range )
    xc=np.mean(xs)
    yc=np.mean(ys)

    # From the array 'triangle', points that lie outside the triangle will be
    # set to 'False'.
    triangle = np.ones(X.shape,dtype=bool)
    for i in range(3):
        ii=(i+1)%3
        if xs[i]==xs[ii]:
            include = X *(xc-xs[i])/abs(xc-xs[i]) > xs[i] *(xc-xs[i])/abs(xc-xs[i])
        else:
            poly=np.poly1d([(ys[ii]-ys[i])/(xs[ii]-xs[i]),ys[i]-xs[i]*(ys[ii]-ys[i])/(xs[ii]-xs[i])])
            include = Y *(yc-poly(xc))/abs(yc-poly(xc)) > poly(X) *(yc-poly(xc))/abs(yc-poly(xc))
        triangle*=include

    # Output: 2 arrays with the x- and y- coordinates of the points inside the
    # triangle.
    

    x_points = X[triangle]
    y_points = Y[triangle]

    points = np.column_stack((x_points.astype(int), y_points.astype(int)))

    itbuffer = np.empty(shape=((points.size/2),4), dtype=np.int32)
    itbuffer.fill(np.nan)

    itbuffer[:,0] = x_points
    itbuffer[:,1] = y_points
    itbuffer[:,2] = map_np[points[:,1],points[:,0]]

# Get distance values    

    deltax = itbuffer[:,0]-robot_x
    deltay = itbuffer[:,1]-robot_y
    
    deltax = map_resolution*deltax
    deltay = map_resolution*deltay
    
    
    deltax = np.square(deltax)
    deltay = np.square(deltay)

    
    dist_sq = deltax+deltay
    
    dist_sq = np.sqrt(dist_sq)
#     dist_sq = map_resolution*dist_sq
    
    itbuffer[:,3] = dist_sq
#     print robot_x,robot_y
#     print np.column_stack((deltax.astype(int),deltay.astype(int)))
#     print time.time()-start_time, "inside_triangle"
    return itbuffer

In [ ]:
def get_points(buoy_yaw, index):
    start_time = time.time()
    global robot_x
    global robot_y
    global camera_range
    global tri_width
    global areas
    global dist_error
    
    appr_dist_up = (1.0+2.0*dist_error)*2*456.7946397925*math.pow(areas[index],-0.4705300238)
    point_deltax = appr_dist_up * math.sin(math.radians(buoy_yaw)) # appr_dist_up
    point_deltay = appr_dist_up * math.cos(math.radians(buoy_yaw)) #appr_dist_up
    per_slope = -1.0/(point_deltay/point_deltax)
    point1 = np.array([round(robot_x), round(robot_y)], dtype=int)
    point2 = np.array([round(robot_x+point_deltax), round(robot_y+point_deltay)], dtype=int)

    
    
    point3_x = (robot_x+point_deltax) + math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))
    point4_x = (robot_x+point_deltax) - math.sqrt(math.pow(tri_width,2)/(math.pow(per_slope,2)+1))

    point3_y = robot_y+point_deltay + per_slope*(point3_x-(robot_x+point_deltax))
    point4_y = robot_y+point_deltay + per_slope*(point4_x-(robot_x+point_deltax))

    point3 = np.array([round(point3_x),round(point3_y)], dtype=int)
    point4 = np.array([round(point4_x),round(point4_y)], dtype=int)
    
#     print time.time()-start_time, "get_points"
    return point1,point2,point3,point4

In [ ]:
def resize(map_image, (width,height), (origin_xdesired, origin_ydesired)):
    
    global origin
    
    # print map_image.shape, "original"
    
    # This adds padding to the left side of the map (map is oriented with shoreline on the bottom)
    # if the origin is less than the desired amount in the x direction
    if (-origin.x) < origin_xdesired:
        left_padding = np.zeros(((origin_xdesired+int(origin.x)),map_image.shape[1]))
        map_image = np.vstack((left_padding, map_image))
    # print map_image.shape, "left_padding"
    # This deletes part of the left side of the map if the origin is greater than the desired amount
    # is the x direction
    if (-origin.x) > origin_xdesired:
        map_image = map_image[:,-int(origin.x + origin_xdesired):]
    # print map_image.shape, "left_padding"
    if (-origin.y) < origin_ydesired:
        top_padding = np.zeros((map_image.shape[0],(origin_ydesired+int(origin.y))))             
        map_image = np.hstack((top_padding, map_image))
    # print map_image.shape, "top_padding"
    # This deletes part of the top of the map if the origin is greater than the desired amount
    # is the y direction
    if (-origin.y) > origin_ydesired:
        map_image = map_image[-int(origin.y + origin_ydesired):,:]
    if map_image.shape[0] < width:
        right_padding = np.zeros(((width-map_image.shape[0]),map_image.shape[1]))
        # print right_padding.shape
        map_image = np.vstack((map_image, right_padding))
    # print map_image.shape, "right_padding"
    if map_image.shape[1] < height:
        bottom_padding = np.zeros((map_image.shape[0],(height-map_image.shape[1])))
        # print bottom_padding.shape
        map_image = np.hstack((map_image, bottom_padding))
    # print map_image.shape, "bottom_padding"
    
    map_image = map_image[0:width][0:height]
    up_occ_map.info.width = map_image.shape[0]
    up_occ_map.info.height = map_image.shape[1]
    
    up_occ_map.info.origin.position.x = -origin_ydesired*map_resolution
    up_occ_map.info.origin.position.y = -origin_xdesired*map_resolution
     
    return map_image

In [ ]:
def map_filter(trans, robot_yaw):
    start_time = time.time()
    global origin
    global robot_x
    global robot_y
    global camera_range
    global occ_map
    global bearings
    global display_robot_yaw
    global tags
    
    map_np, robot_yaw = get_mapnbot_info(trans,robot_yaw) 
    
    slices = []
    i = 0
    while i < len(bearings):
        
        
        buoy_yaw = robot_yaw + bearings[i]
        if buoy_yaw < 0:
            buoy_yaw = buoy_yaw + 360
#         print bearings
##############################################################################
# This is for visualization purposes.
# Set to false if you don't want to display the robot_yaw
# Delete it if it's irritating to look at
        if display_robot_yaw and occ_map.header.seq % 5 == 0:
            bearings.append(0.0)
            print "Robot_yaw: ", robot_yaw
            print "Buoy yaw: ", buoy_yaw
            print len(slices)
##############################################################################
        
        pt1, pt2, pt3, pt4 = get_points(buoy_yaw, i)    
        
#         sliced = createLineIterator(pt1.astype(int),pt2.astype(int),map_np)

        sliced = insidetriangle((pt3[0],pt1[0],pt4[0]),(pt3[1],pt1[1],pt4[1]),map_np)
        slices.append(np.array(sliced).astype(int))
#         print slices
        i+=1
#     print time.time()-start_time, "map_filter"
    return slices

In [ ]:
def resize_visualization(map_image, (origin_xdesired, origin_ydesired)):
    
    left = 0.0
    top = 0.0
    bottom = 0.0
    right = 0.0
    if (-origin.x) < origin_xdesired:
        left = origin_xdesired+origin.x
    if (-origin.y) < origin_ydesired:
        top = origin_ydesired+origin.y
    if map_image.shape[0] < 500:
        bottom = 500-map_image.shape[0]
    if map_image.shape[1] < 500:
        right = 500-map_image.shape[1]
  
    cv.copyMakeBorder(map_image,int(top),int(bottom),int(left),int(right),cv.BORDER_CONSTANT,value=[255,255,255])
    return map_image[0:500][0:500]
    

In [ ]:
def get_map(slices):
    start_time = time.time()
    global camera_range
    global origin
    global robot_x
    global robot_y
    global map_width
    global map_height
    global occ_map
    global visualization

    
    map_np = np.reshape(np.array(occ_map.data),(map_height,map_width))
    
    if visualization:
        map_img_z = np.zeros((map_height,map_width,3))
    else:
        map_img_z = np.zeros((map_height, map_width))
    
    map_img_b = draw_background_objects(map_img_z,map_np)
           
    map_img_r = draw_ray(map_img_b,slices)
    
    # if not visualization:
    #     map_img_r = resize(map_img_r, (500,500),(100,30))
    # else:
    #     map_img_r = resize_visualization(map_img_r, (100,30))

    return map_img_r

In [ ]:
def create_array(map_image):
    
    global map_width
    global map_height
    global visualisation
    
    my_array = Array(data=np.array(map_image).flatten().astype(np.int32).tolist())
    
    if visualization:
         my_array.layout.dim = [Dimension("height",map_height,1*map_width*map_height),Dimension("width",map_width,1*map_width),Dimension("channel",3,3)]
    else:
        my_array.layout.dim = [Dimension("height",map_height,1*map_width*map_height),Dimension("width",map_width,1*map_width)]
    
    return my_array
    

In [ ]:
def callback(rect, p_map):
    
    start_time = time.time()
    
    global occ_map
    global origin
    global bearings
    global listener 
    global display_rect
    global display_map
    global tags
    global up_occ_map
    global pub_array
    global pub_map
    global map_height
    global map_width
    global b_list
    global visualization
    global publish_map_or_nah

    occ_map = p_map    
    up_occ_map = p_map
    
    origin = occ_map.info.origin.position
    rect = transform(rect)
    post = time.time()
    
    if not rospy.is_shutdown():
        
        now = rospy.Time.now()
        listener.waitForTransform("/odom", "/front_left_camera_link", now, rospy.Duration(1))
        (trans,rot) = listener.lookupTransform("/odom", "/front_left_camera_link", now)
        
        euler = tf.transformations.euler_from_quaternion(rot)
        pixel_lines = map_filter(trans, math.degrees(euler[2]))
        map_image = get_map(pixel_lines) 
        
        
        if publish_map_or_nah and not visualization: 
            
            map_obj = EnvMap(map_image,[int(origin.y),int(origin.x)]) 
                
            buoy_data = map_obj.get_buoys()
            
            for item in buoy_data:
                b_list.add_buoy(item)
            map_obj.set_buoys(b_list.get_confirmed_buoys())
            map_image = map_obj.get_map()
            
            
            up_occ_map.data = np.array(map_image).flatten().astype(np.int32)

            if pub_map.get_num_connections > 0:
                pub_map.publish(up_occ_map)
        
        else:
 
            my_array_for_publishing = create_array(map_image)
            pub_array.publish(my_array_for_publishing)
            
        if display_rect:
            disp_rect(rect)
        if display_map and visualization:
            disp_map(map_image)
            
    # print post-start_time, "transform"
    # print time.time()-post, "post"
    # print time.time()-start_time, "callback", "\n\n"

In [ ]:
if __name__ == '__main__':   
    
    global b_list
    
    b_list = BuoyList()
    rect = message_filters.Subscriber("stereo/left/image_rect_color", Image)
    p_map = message_filters.Subscriber("projected_map", Map)
    
    ts = message_filters.ApproximateTimeSynchronizer([rect, p_map], 10, 0.5)
    ts.registerCallback(callback)
    rospy.spin()  